In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Tue Mar  7 13:35:54 2023'

In [2]:
import torch
from torch import nn
a = nn.Linear(128,128)
b = torch.empty((2,128,128))
a(b).shape

torch.Size([2, 128, 128])

In [3]:
import numpy as np
np.sort(np.random.choice([1,3,5,7,9], 5 , replace = False))

array([1, 3, 5, 7, 9])

In [1]:
from utils.dataset import read_pic
import numpy as np
import cv2
p = read_pic("/home/supercgor/gitfile/data/bulkice/afm/T160_1", transform = True, rec_size= 0.02)
p.size()
p = p * 255
p = p.numpy()
p.astype(np.uint8)
p = np.reshape(p, (5,4, 128, 128))
out = np.concatenate(p, axis = 1)
out = np.transpose(out, (0,2,1))
out = np.concatenate(out, axis = 0)
cv2.imwrite("test2.png", out)

True